# Scene Boundary Paritioning
Movies are comprised of self-contained units called scenes. Scenes have a beginning and end, usually comprised of a single conversation. They most often take place in one location with a fixed number of characters. By identifying scenes in a movie, we can then begin to analyze them individually, most notably by treating a scenes's dialogue as a freestanding, indepdent conversation.

To start, we'll just be identifying two-character dialogue scenes. These are the most basic building-blocks of films: just two characters speaking together with no distractions, purely advancing the plot with their dialogue. In modern filmmaking, these scenes are usually shot in a specific manner. We can take advantage of this by looking for specific patterns of shots, to identify a few two-character dialogue scenes.

In [1]:
import sys
sys.path.append('../data_serialization')
from serialization_preprocessing_io import *
from time_reference_io import *
from scene_identification_io import *

We have saved pickle objects of various dataframes. We'll load into memory the five dataframes, but we're most interested in the two which deal with onscreen images. The each have one row per frame (screencap), with one frame per second — so each row represents one second of onscreen action.
- vision_df: contains general computer vision information on each frame, including clusterings of similar frames into "shots"
- face_df: contains information related to faces found, including their vectorized encodings, and clusters of these encodings

In [2]:
film = 'lost_in_translation_2003'
srt_df, subtitle_df, sentence_df, vision_df, face_df = read_pickle(film)

## The A/B/A/B pattern
In modern film, two-character dialogue scenes follow a very distinct pattern. Character A speaks, then Character B, then back to A, then to B, etc. We cut back and forth between the two characters.


### Anchor Shots
We look for these two Anchor shots, which are the shots of the two characters and form the A/B/A/B pattern. We'll be looking through every frame in the film, and trying to find these ABAB patterns.

The key to this lies in two columns in vision_df:
- shot_cluster:  represents clusters of similar frames, or shots. Think of a four-second shot of a character speaking. This would be represented as four rows with a common shot_cluster
- shot_id: sequential numbering of each shot (regardless of uniqueness). Every time a shot changes (and even if we've seen this shot before), the shot_id is incremented by 1

In [3]:
vision_df[202:213]

,blank,aspect_ratio,brightness,contrast,blue,green,red,shot_cluster,shot_id
frame,,,,,,,,,
203,None,1.84,26,20,21,23,34,275,37
204,None,1.84,26,20,21,23,34,275,37
205,None,1.84,36,28,43,35,35,3,38
206,None,1.84,39,28,47,39,38,3,38
207,None,1.84,38,29,46,37,37,3,38
208,None,1.84,36,29,44,36,34,3,38
209,None,1.84,33,28,40,32,32,3,38
210,None,1.84,32,30,38,32,32,3,38
211,None,1.84,32,29,37,31,31,3,38


The below code will generate two lists each time an ABAB pattern is found:
- alternating_pairs: the two shot_clusters
- pair_shot_ids: the beginning and ending shot_id

In [4]:
shot_id_list = vision_df.shot_id.tolist()
shot_clusters = vision_df.shot_cluster.tolist()
frame_choice = range(1, (len(vision_df) + 1))

# to check for an A/B/A/B pattern, we must store the previous three clusters in memory
prev_clust_1 = 1001
prev_clust_2 = 1002
prev_clust_3 = 1003
prev_shot_id = -1
alternate_a_list = []
alternate_b_list = []
pair_shot_ids = []
pair_found = 0

# zip our various lists into a usable data structure
for frame_file, cluster, shot_id in zip(frame_choice, shot_clusters, shot_id_list):


    # we use prev_shot_id to identify when there's a new shot (when the cluster value changes)
    # when iterating through each frame, look for an A/B/A/B pattern, and save the clusters of any patterns
    if shot_id != prev_shot_id:
        if cluster == prev_clust_2 and prev_clust_1 == prev_clust_3:
            if pair_found == 0:
                alternate_a_list.append(min(cluster, prev_clust_1)) # min and max are used to avoid duplicates of (1, 2), (2, 1)
                alternate_b_list.append(max(cluster, prev_clust_1))
                beginning_shot = shot_id - 3
            pair_found = 1
        else:
            if pair_found == 1:
                ending_shot = shot_id - 1
                pair_shot_ids.append([beginning_shot, ending_shot])
            pair_found = 0
        
        # every time there's a new shot, we update the cluster memory
        prev_shot_id = shot_id
        prev_clust_3 = prev_clust_2
        prev_clust_2 = prev_clust_1
        prev_clust_1 = cluster
        
    # the below print can be used for troubleshooting and visualizing the memory state at each frame
    # print(frame_file, '\t', mcu_flag, '\t', cluster,'\t', shot_id, '\t', prev_shot_id, '\t', prev_clust_1, '\t', prev_clust_2, '\t', prev_clust_3, '\tend')

# save non-unique alternating pairs, because these must line up with pair_shot_ids
alternating_pairs = []

for a, b, in zip(alternate_a_list, alternate_b_list):
    alternating_pairs.append([int(a), int(b)])

print(len(alternating_pairs))
print(len(pair_shot_ids))

58
58


In [5]:
alternating_pairs[0]

[0, 151]

In [6]:
pair_shot_ids[0]

[57, 61]